In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 100  # number of expression blocks
num_data = 9100   # number of data
test_size = 100   # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100   # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 50                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 26030.72
Epoch 1, Validation Loss: 2930.95
Epoch 2, Validation Loss: 1333.36
Epoch 3, Validation Loss: 1246.20
Epoch 4, Validation Loss: 1659.47
Epoch 5, Validation Loss: 1172.19
Epoch 6, Validation Loss: 1022.68
Epoch 7, Validation Loss: 946.34
Epoch 8, Validation Loss: 966.85
Epoch 9, Validation Loss: 864.04
Epoch 10, Validation Loss: 906.02
Epoch 11, Validation Loss: 746.60
Epoch 12, Validation Loss: 658.56
Epoch 13, Validation Loss: 598.23
Epoch 14, Validation Loss: 735.98
Epoch 15, Validation Loss: 583.67
Epoch 16, Validation Loss: 847.25
Epoch 17, Validation Loss: 659.72
Epoch 18, Validation Loss: 635.99
Epoch 19, Validation Loss: 668.71
Epoch 20, Validation Loss: 1182.84
Epoch 21, Validation Loss: 624.68
Epoch 22, Validation Loss: 608.55
Epoch 23, Validation Loss: 587.39
Epoch 24, Validation Loss: 575.50
Epoch 25, Validation Loss: 562.22
Epoch 26, Validation Loss: 517.39
Epoch 27, Validation Loss: 591.82
Epoch 28, Validation Loss: 493.23
Epoch 29, Valid

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-100_g.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 52.53it/s]


           Obj Val  Constraints Viol  Elapsed Time
count   100.000000             100.0    100.000000
mean    519.848639               0.0      0.003384
std     171.564230               0.0      0.001033
min     340.440396               0.0      0.001996
25%     409.692135               0.0      0.002559
50%     467.591698               0.0      0.003038
75%     599.028844               0.0      0.004056
max    1325.291318               0.0      0.006001
Number of infeasible solution: 0


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
growth_rate = 1.05              # growth rate of penalty weight
warmup = 50                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, growth_rate, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 22853.77
Epoch 1, Validation Loss: 2533.40
Epoch 2, Validation Loss: 1863.50
Epoch 3, Validation Loss: 1134.42
Epoch 4, Validation Loss: 1130.99
Epoch 5, Validation Loss: 952.80
Epoch 6, Validation Loss: 795.46
Epoch 7, Validation Loss: 741.50
Epoch 8, Validation Loss: 1215.76
Epoch 9, Validation Loss: 694.71
Epoch 10, Validation Loss: 703.77
Epoch 11, Validation Loss: 635.50
Epoch 12, Validation Loss: 626.58
Epoch 13, Validation Loss: 616.27
Epoch 14, Validation Loss: 667.66
Epoch 15, Validation Loss: 665.05
Epoch 16, Validation Loss: 744.93
Epoch 17, Validation Loss: 657.28
Epoch 18, Validation Loss: 655.65
Epoch 19, Validation Loss: 722.54
Epoch 20, Validation Loss: 645.25
Epoch 21, Validation Loss: 768.43
Epoch 22, Validation Loss: 649.91
Epoch 23, Validation Loss: 730.92
Epoch 24, Validation Loss: 642.45
Epoch 25, Validation Loss: 636.79
Epoch 26, Validation Loss: 1812.16
Epoch 27, Validation Loss: 1492.86
Epoch 28, Validation Loss: 658.68
Epoch 29, Valid

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
df.to_csv("result/rb_lr_50-100_g.csv")
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 61.75it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   618.512688               0.0      0.003061
std    104.892178               0.0      0.000965
min    361.382483               0.0      0.001889
25%    555.203302               0.0      0.002077
50%    624.097104               0.0      0.003000
75%    699.911344               0.0      0.003557
max    820.812557               0.0      0.006630
Number of infeasible solution: 0
